In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
a1 = 0.0134
b1 = 1
c1 = 4.35e-4
m1 = 1
u0 = 300
a = 10
b = 10
x0 = 5
z0 = 5
alpha_1 = 0.05
alpha_2 = 0.1
alpha_3 = 0.5
alpha_4 = 1.0
f0 = 1
beta = 1
F0 = 30

In [5]:
def der_lambda(y):
    return a1 * c1 * m1 * (y ** (m1 - 1))

def _lambda(y):
    return a1 * (b1 + c1 * (y ** m1))

def _f(x, z):
    return f0 * np.exp(-beta * ((x - x0) ** 2 + (z - z0) ** 2))

def kappa(y1, y2):
    return (_lambda(y1) + _lambda(y2)) / 2

In [6]:
def x_left_bound_cond():
    xk0 = 1
    xm0 = 0
    xp0 = 0
    return xk0, xm0, xp0

def x_right_bound_cond():
    xkN = 0
    xmN = 1
    xpN = 0
    return xkN, xmN, xpN

def x_right_sweep(curr_z, y_prev, y_curr, x0, xN, h_x, tau):
    eps = 1e-4
    flag = True
    prev_res = y_curr
    y = []
    
    while flag:
        k0, m0, p0 = x_left_bound_cond()
        kN, mN, pN = x_right_bound_cond()
        
        ksi = [0, -m0 / k0]
        eta = [0, p0 / k0]
        x = h_x
        n = 1
        
        for i in range(1, len(y_curr) - 1):
            aN = tau / (h_x ** 2) * kappa(y_curr[i - 1], y_curr[i])
            dN = tau / (h_x ** 2) * kappa(y_curr[i], y_curr[i + 1])
            bN = tau / (h_x ** 2) + aN + dN
            fN = _f(x, curr_z) * tau / 2 + y_prev[i]
            
            aN_1 = tau / (2 * h_x**2) * der_lambda(y_curr[i - 1]) * (y_curr[i - 1] - y_curr[i]) + aN
            bN_1 = tau / (2 * h_x**2) * der_lambda(y_curr[i]) * (-y_curr[i - 1] + 2 * y_curr[i] - y_curr[i + 1]) + bN
            dN_1 = tau / (2 * h_x**2) * der_lambda(y_curr[i + 1]) * (-y_curr[i] + y_curr[i + 1]) + dN
            fN_1 = aN * y_curr[i - 1] - bN * y_curr[i] + dN * y_curr[i + 1] + fN
            
            ksi.append(dN_1 / (bN_1 - aN_1 * ksi[n]))
            eta.append((aN_1 * eta[n] + fN_1) / (bN_1 - aN_1 * ksi[n]))
            
            n += 1
            x += h_x
        
        y = [0] * (n + 1)
        y[n] = (pN - kN * eta[n]) / (kN * ksi[n] + mN)
        
        for i in range(n - 1, -1, -1):
            y[i] = ksi[i + 1] * y[i + 1] + eta[i + 1]
            
        cnt = 0
        for i in range(n + 1):
            if abs((y[i] - prev_res[i]) / y[i]) < eps:
                cnt += 1
                
        if cnt == len(y):
            flag = False
            
        prev_res = y
    
    return y

def z_left_bound_cond():
    zk0 = 1
    zm0 = 0
    zp0 = 0
    return zk0, zm0, zp0

def z_right_bound_cond():
    zkN = 1
    zmN = 0
    zpN = 0
    return zkN, zmN, zpN

def z_right_sweep(curr_x, y_prev, y_curr, z0, zM, h_z, tau):
    eps = 1e-4
    flag = True
    prev_res = y_curr
    y = []
    
    while flag:
        k0, m0, p0 = z_left_bound_cond()
        kN, mN, pN = z_right_bound_cond()
        
        ksi = [0, -m0 / k0]
        eta = [0, p0 / k0]
        z = h_z
        n = 1
        
        for i in range(1, len(y_curr) - 1):
            aN = tau / (h_z ** 2) * kappa(y_curr[i - 1], y_curr[i])
            dN = tau / (h_z ** 2) * kappa(y_curr[i], y_curr[i + 1])
            bN = tau / (h_z ** 2) + aN + dN
            fN = _f(z, curr_x) * tau / 2 + y_prev[i]
            
            aN_1 = tau / (2 * h_z**2) * der_lambda(y_curr[i - 1]) * (y_curr[i - 1] - y_curr[i]) + aN
            bN_1 = tau / (2 * h_z**2) * der_lambda(y_curr[i]) * (-y_curr[i - 1] + 2 * y_curr[i] - y_curr[i + 1]) + bN
            dN_1 = tau / (2 * h_z**2) * der_lambda(y_curr[i + 1]) * (-y_curr[i] + y_curr[i + 1]) + dN
            fN_1 = aN * y_curr[i - 1] - bN * y_curr[i] + dN * y_curr[i + 1] + fN
            
            ksi.append(dN_1 / (bN_1 - aN_1 * ksi[n]))
            eta.append((aN_1 * eta[n] + fN_1) / (bN_1 - aN_1 * ksi[n]))
            
            n += 1
            z += h_z
        
        y = [0] * (n + 1)
        y[n] = (pN - kN * eta[n]) / (kN * ksi[n] + mN)
        
        for i in range(n - 1, -1, -1):
            y[i] = ksi[i + 1] * y[i + 1] + eta[i + 1]
            
        cnt = 0
        for i in range(n + 1):
            if abs((y[i] - prev_res[i]) / y[i]) < eps:
                cnt += 1
                
        if cnt == len(y):
            flag = False
            
        prev_res = y
    
    return y

In [2]:
def is_fault_zero(y_delta, cur_y, eps):
    cnt = 0
    for i in range(len(y_delta)):
        if abs(y_delta[i] / cur_y[i]) < eps:
            cnt += 1
    
    if cnt == len(y_delta):
        return True
    
    return False

def get_approx_1(approx_0, curr_z, x0, xN, h_x, tau):
    eps = 1e-4
    curr_approx_1 = approx_0.copy()
    fault_flag = True
    
    while fault_flag:
        curr_approx_delta_1 = x_right_sweep(curr_z, approx_0, curr_approx_1, x0, xN, h_x, tau)
        next_approx_1 = curr_approx_1 + curr_approx_delta_1
        fault_flag = is_fault_zero(curr_approx_delta_1, next_approx_1, eps)
        curr_approx_1 = next_approx_1
    
    return curr_approx_1

def get_approx_2(approx_1, curr_x, z0, zM, h_z, tau):
    eps = 1e-4
    prev_approx_2 = []
    curr_approx_2 = []
    fault_flag = True
    
    while fault_flag:
        curr_approx_delta_2 = z_right_sweep(curr_x, prev_approx_2, approx_1, z0, zM, h_z, tau)
        curr_approx_2 = prev_approx_2 + curr_approx_delta_2
        fault_flag = is_fault_zero(curr_approx_delta_2, prev_approx_2, eps)
        prev_approx_2 = prev_approx_2 + curr_approx_delta_2
    
    return curr_approx_2

In [3]:
def get_layer_solution(approx_0, x0, xN, h_x, z0, zM, h_z, tau):
    # добавить цикл по z и x
    approx_1 = get_approx_1(approx_0, curr_z, x0, xN, h_x, tau)
    approx_2 = get_approx_2(approx_1, curr_x, z0, zM, h_z, tau)
    return approx_2

def is_left_side_zero(y_t_m, y_t_m_1, eps):
    cnt = 0
    for i in range(len(y_t_m)):
        if abs((y_t_m[i] - y_t_m_1[i]) / y_t_m_1[i]) < eps:
            cnt += 1
    
    if cnt == len(y_t_m):
        return True
    
    return False

def get_solution(x0, xN, h_x, z0, zM, h_z, tau):
    eps = 1e-4
    approx_0 = []
    prev_res = approx_0.copy()
    res = []
    left_side_zero = True
    
    while not left_side_zero:
        res = get_layer_solution(prev_res, x0, xN, h_x, z0, zM, h_z, tau)
        left_side_zero = is_left_side_zero(prev_res, res, eps)
        prev_res = res
        
    return res